<a href="https://colab.research.google.com/github/jgp20/HCSPD2024/blob/main/Pruebas_Clase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actualizar librerias - instalación

In [3]:
!apt-get update #Usar para actualizar librerias, únicamente

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [4]:
!apt install octave #Instala colab en el entorno de trabajo

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
octave is already the newest version (6.4.0-2).
0 upgraded, 0 newly installed, 0 to remove and 58 not upgraded.


In [5]:
# https://gnu-octave.github.io/packages/
# https://github.com/gnu-octave/symbolic
!apt install octave python3-sympy #Instala el paquete simbólico de Octave, que incluye el Sympy de Python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
octave is already the newest version (6.4.0-2).
The following additional packages will be installed:
  libimagequant0 libraqm0 mailcap mime-support python3-mpmath python3-numpy python3-olefile
  python3-pil
Suggested packages:
  python-mpmath-doc python3-gmpy2 python3-matplotlib python-numpy-doc python3-pytest python-pil-doc
  texlive-fonts-extra dvipng python-sympy-doc
The following NEW packages will be installed:
  libimagequant0 libraqm0 mailcap mime-support python3-mpmath python3-numpy python3-olefile
  python3-pil python3-sympy
0 upgraded, 9 newly installed, 0 to remove and 58 not upgraded.
Need to get 8,725 kB of archives.
After this operation, 54.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libimagequant0 amd64 2.17.0-1 [34.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libraqm0 amd64 0.7.0-4ubuntu1 [11.7 kB]
Get:3

In [6]:
# Writing Inst_S.m
%%writefile Inst_S.m
pkg install -forge symbolic

Writing Inst_S.m


In [7]:
!octave -W Inst_S.m #  corre el código adentro de Octave pkg install -forge symbolic

For information about changes from previous versions of the symbolic package, run 'news symbolic'.


In [10]:
!pip install --user sympy==1.5.1 #Normalmente no es necesario, pero ésta versión actual de "octave-symbolic" lo exige



Se debe agregar en el .m el path donde está el Python, mediante una línea como ésta:

setenv PYTHON /usr/local/bin/python

para acceder al sympy desde symbolic. No obtante, puede ser diferente el path y debe verificarse en caso de que no funcione.


In [9]:
%%writefile Graficos_Octave.m
available_graphics_toolkits()

Writing Graficos_Octave.m


In [30]:
!octave -W Graficos_Octave.m

ans =
{
  [1,1] = gnuplot
}



#Cálculo simbólico circuito RLC - 1

In [11]:
%%writefile Calculo_RLC_Simbolico.m
clear all; clc;
setenv PYTHON /usr/local/bin/python %Acá está el vinculo con Pyton, para acceder al sympy desde symbolic
%/usr/bin/python
pkg load symbolic %Primera vez que se corre el paquete simbólico de Python.
% #R=2.2e3;L=10e-6;C=100e-9;
syms ii(t) v(t)
ode2=diff(v)==ii;
% ode1=diff(ii)==-R/L*ii-1/L*v+1/L*heaviside(0);
ode1=diff(ii)==-ii-v+heaviside(0,1);%heaviside(0) devuelve .5
cond1 = ii(0) == 0;
cond2 = v(0) == 0;
conds = [cond1;cond2];
odes = [ode1; ode2];
Sol=dsolve(odes,ii(0) == .0,v(0) == .0);
Corriente=simplify(Sol.ii)
Tension=simplify(Sol.v)

Writing Calculo_RLC_Simbolico.m


In [12]:
!octave -W Calculo_RLC_Simbolico.m # -W : no window system

Symbolic pkg v3.1.1: Python communication link active, SymPy v1.5.1.
Corriente = (sym)

        -t           
        ───          
         2     ⎛√3⋅t⎞
  2⋅√3⋅ℯ   ⋅sin⎜────⎟
               ⎝ 2  ⎠
  ───────────────────
           3         

Tension = (sym)

            -t               
            ───              
             2     ⎛√3⋅t   π⎞
      2⋅√3⋅ℯ   ⋅sin⎜──── + ─⎟
                   ⎝ 2     3⎠
  1 - ───────────────────────
                 3           




In [15]:
%%writefile Solucion_analitica.m
clc;clear all;
warning('off','all');
color_='r';
color='b';
X=-[0; 0];ii=0;t_etapa=1e-8;wRef=2;tF=1e-3;
Ts=t_etapa;
t=0:t_etapa:tF;
x1_analitica = (2.*sqrt(3) .* exp(-t/2) .* sin ( sqrt(3).*t / 2))/3
x2_analitica = 1-(2.*sqrt(3) .* exp(-t/2) .* sin ( sqrt(3).*t / 2))/3
hfig1 = figure(1); set(hfig1, 'Visible', 'off');
subplot(3,1,1);hold on;
%plot(t,x1,color_);title('x_1 corriente'); hold on;
plot(t,x1_analitica,color);title('x_1 corriente'); hold on;
subplot(3,1,2);hold on;
%plot(t,x2,color_);title('x_2 Vc_t');
plot(t,x1_analitica,color);title('x_1 corriente'); hold on;
subplot(3,1,3);hold on;
plot(t,acc,color_);title('Entrada u_t, v_a');
xlabel('Tiempo [Seg.]');
%figure(1)
%figure(2)
print(hfig1,'Vars_temporal','-r300','-dpng');
save('Solucion_analitica_1.mat','-v7');

Overwriting Solucion_analitica.m


In [16]:
# -W : no window system
!octave -W Solucion_analitica.m
import scipy.io
mat = scipy.io.loadmat('/content/Solucion_analitica_1.mat')

Se truncaron las últimas líneas 5000 del resultado de transmisión.

 Columns 90017 through 90024:

   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991

 Columns 90025 through 90032:

   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991

 Columns 90033 through 90040:

   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991

 Columns 90041 through 90048:

   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991

 Columns 90049 through 90056:

   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991

 Columns 90057 through 90064:

   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991

 Columns 90065 through 90072:

   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991

 Columns 90073 through 90080:

   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991

 Columns 90081 through 90088:

   0.9991   0.9991   0.9991   0.9991   0.9991   0.9991   0.99

FileNotFoundError: [Errno 2] No such file or directory: '/content/Solucion_analitica_1.mat'

In [ ]:
from PIL import Image
imt = Image.open("/content/Vars_temporal.png")


# Verificación numérica - 1

In [21]:
%%writefile Verificacion_Num.m
clc;clear all;
warning('off','all');
X=-[0; 0];ii=0;t_etapa=1e-8;wRef=2;tF=1e-3;
color_='r';
color='b';
Ts=t_etapa;
%A1=((2*Kp*Ts)+(Ki*(Ts^2))+(2*Kd))/(2*Ts);
%B1=(-2*Kp*Ts+Ki*(Ts^2)-4*Kd)/(2*Ts);
%C1=Kd/Ts;
%e=zeros(round(tF/t_etapa),1);
u=0;TL=1.15e-4;u=12;
%graphics_toolkit("gnuplot"); %No aconsejado
%graphics_toolkit("fltk");
%graphics_toolkit("qt"); %Aconsejado
% x1(1)=X(1);%corriente
% x2(2)=X(2);%tensión en el capacitor
for t=0:t_etapa:tF
  ii=ii+1;k=ii+2;
  x1(ii)=X(1);%corriente
  x2(ii)=X(2);%tensión en el capacitor
  X=modrlc(t_etapa, X, u);
  acc(ii)=u;
end
t=0:t_etapa:tF;
hfig1 = figure(1); set(hfig1, 'Visible', 'off');
subplot(3,1,1);hold on;
plot(t,x1,color_);title('x_1 corriente'); hold on;
%plot(t,x1_analitica,color);title('x_1 corriente'); hold on;
subplot(3,1,2);hold on;
plot(t,x2,color_);title('x_2 Vc_t');
subplot(3,1,3);hold on;
plot(t,acc,color_);title('Entrada u_t, v_a');
xlabel('Tiempo [Seg.]');
%figure(1)
%figure(2)
print(hfig1,'Vars_temporal','-r300','-dpng');
save('Verificacion_Num_1.mat','-v7');

Writing Verificacion_Num.m


In [22]:
%%writefile modrlc.m
%Funcion modelo
function [X]=modrlc(t_etapa, xant, accion)
h=1e-9;t_simul=1e-3;R=2.2e3;L=10e-6;C=100e-9;
A=[-R/L, -1/L;1/C,0];B=[1/L;0];%
% eig(A)
% 1./ans
C=[R 0];u=accion;
x=xant;
for ii=1:t_etapa/h
xp=A*x+B*u;
x=x+xp*h;
end
X=[x];%x1 corriente, x2 tensión

Writing modrlc.m


# Graficos

In [ ]:
# -W : no window system
!octave -W Verificacion_Num.m
import scipy.io
mat = scipy.io.loadmat('/content/Verificacion_Num_1.mat')

In [ ]:
from PIL import Image
imt = Image.open("/content/Vars_temporal.png")
imt

In [ ]:
from google.colab import drive #ésto es para guardar los resultados en el Gdrive.
drive.mount('/content/drive')

In [ ]:
# !cp Vars_temporal.png /content/drive/MyDrive/CursoHCSPD2024 #Copia uno por uno
import os
import shutil
fnames = ['Vars_temporal.png','Calculo_RLC_Simbolico.m','Verificacion_Num.m','Verificacion_Num_1.mat','modrlc.m'] #Acá van los nombres de los archivos que desean copiarse al Gdrive
for fname in fnames:
  src = os.path.join('/content', fname)
  dst = os.path.join('/content/drive/MyDrive/CursoHCSPD2024', fname)
  shutil.copyfile(src, dst)